In [1]:
import numpy as np
import matplotlib as plt
import random
import cirq

#### a) n = 2

In [3]:
# Input qubits
x0 = cirq.NamedQubit("x0")
x1 = cirq.NamedQubit("x1")
y = cirq.NamedQubit("y")
func = np.array(((1, 0), (0, 1)))
print('Secret function: f(0,0)={}, f(0,1))={}, f(1,0)={}, f(1,1)={}\n'.format(
    func[0,0], func[0,1], func[1,0], func[1,1]))

# Gates
ops = [cirq.H(x0), cirq.H(x1), cirq.X(y), cirq.H(y)]

# Oracles
if not func.any(): # f = 0
    ops += []
elif func.all(): # f = 1
    ops += [cirq.X(y)]

elif func[0,0] == func[1,0]: # f(*,x0) = 1
    if func[0,0] == 1:
        ops += [cirq.X(x0)]
    ops += [cirq.CNOT(x0, y)]

elif func[0,0] == func[0,1]: # f(x1,*) = 1
    if func[0,0] == 1:
        ops += [cirq.X(x1)]
    ops += [cirq.CNOT(x1, y)]

elif func[0,0] == func[1,1]: # f(x1 == x0) = 1
    ops += [cirq.CNOT(x0, y), cirq.CNOT(x1, y)]
    if func[0,0] == 1:
        ops += [cirq.X(y)]
# end of oracles

ops += [cirq.H(x0), cirq.H(x1), cirq.measure(x0, x1)]

# Create circuit
circuit = cirq.Circuit.from_ops(ops)
print(circuit, '\n')

# Simulate circuit
simulator = cirq.Simulator()
result = simulator.run(circuit)
print('Measurement result:', result)

Secret function: f(0,0)=1, f(0,1))=0, f(1,0)=0, f(1,1)=1

x0: ───H───────@───H───────M───
               │           │
x1: ───H───────┼───@───H───M───
               │   │
y: ────X───H───X───X───X─────── 

Measurement result: x0,x1=1, 1


#### b) n = 3

In [4]:
import ps6_U_f

# Input qubits
x0 = cirq.NamedQubit("x0")
x1 = cirq.NamedQubit("x1")
x2 = cirq.NamedQubit("x2")
y = cirq.NamedQubit("y")

# Gates
ops = [cirq.H(x0), cirq.H(x1), cirq.H(x2), cirq.X(y), cirq.H(y)]
# Oracle U_f
ops += ps6_U_f.U_f((x0, x1, x2, y))
ops += [cirq.H(x0), cirq.H(x1), cirq.H(x2), cirq.measure(x0, x1, x2)]

# Create circuit
circuit = cirq.Circuit.from_ops(ops)
print(circuit, '\n')

# Simulate circuit
simulator = cirq.Simulator()
result = simulator.run(circuit)
print('Measurement result:', result)

f=1
x0: ───H───H───M───────────
               │
x1: ───H───H───M───────────
               │
x2: ───H───H───M───────────

y: ────X───H───X───X───X─── 

Measurement result: x0,x1,x2=0, 0, 0


#### c) n qubits

In [6]:
def deutsch_jozsa(qubits, oracle):
    '''Deutsch-Josza-circuit simulator for any unitary oracle enconding a n-bit function.'''
    
    # Create circuit
    circuit = cirq.Circuit()
    
    # Initialize qubits
    for qb in qubits[:-1]:
        circuit.append(cirq.H(qb))
    circuit.append([cirq.X(qubits[-1]), cirq.H(qubits[-1])])
    # Query oracle
    circuit.append(oracle, strategy=cirq.InsertStrategy.NEW_THEN_INLINE)
    # Measure in X basis
    for qb in qubits[:-1]:
        circuit.append(cirq.H(qb), strategy=cirq.InsertStrategy.INLINE)
    circuit.append(cirq.measure(*qubits[:-1]))
    print(circuit, '\n')

    # Simulate circuit
    simulator = cirq.Simulator()
    result = simulator.run(circuit)
    print('Measurement result:', result)
    return circuit

In [5]:
import ps6_U_f
n = 5
qubits = cirq.LineQubit.range(n+1)
circuit = deutsch_jozsa(qubits, ps6_U_f.U_f(qubits))

f=1
0: ───H───────────────────────H───M───
                                  │
1: ───H───────────────────────H───M───
                                  │
2: ───H───────────────────────H───M───
                                  │
3: ───H───────────────────────H───M───
                                  │
4: ───H───────────────────────H───M───

5: ───X───H───X───X───X───X───X─────── 

Measurement result: 0,1,2,3,4=0, 0, 0, 0, 0
